# Create test datasets

Create test datasets that can be used for integration tests for `tiledb_netcdf`, particularly for checking the excessive complexity of `append` (a) works at all, and (b) works correctly for a number of use-cases. Test datasets will be simple NetCDF files with random data and a minimum of required metadata.

## Append use-cases

We want `append` to be able to correctly stitch together multiple datasets in a number of different use-cases:
* single-dim append
* multi-dim append
* scalar dim append
* multiple domains
* combining single-phenomenon datasets
* combinations of the above

## Setup

Imports and functions to make Iris cubes. 

In [ ]:
import iris
from iris.coords import DimCoord
from iris.cube import Cube, CubeList
import netCDF4
import numpy as np

In [ ]:
def make_cube(x_points, y_points, t_points, z_points=None, name="thingness"):
    # Cube data.
    if z_points is not None:
        shape = [len(t_points), len(z_points), len(y_points), len(x_points)]
    else:
        shape = [len(t_points), len(y_points), len(x_points)]
    data = np.zeros(shape)
    
    # Coords.
    x_coord = DimCoord(x_points, standard_name="projection_x_coordinate", units="m")
    y_coord = DimCoord(y_points, standard_name="projection_y_coordinate", units="m")
    t_coord = DimCoord(t_points, standard_name="time", units="hours since epoch")
    if z_points is not None:
        z_coord = DimCoord(z_points, standard_name="height", units="m")
        dcad = [(t_coord, 0), (z_coord, 1), (y_coord, 2), (x_coord, 3)]
    else:
        dcad = [(t_coord, 0), (y_coord, 1), (x_coord, 2)]
        
    # Construct cube.
    cube = Cube(data, long_name=name, units="K",
                dim_coords_and_dims=dcad)
    return cube


def make_scalar_t_cube(x_points, y_points, t_point, name="thingness"):
    # Cube data.
    shape = [len(y_points), len(x_points)]
    data = np.zeros(shape)

    # Coords.
    x_coord = DimCoord(x_points, standard_name="projection_x_coordinate", units="m")
    y_coord = DimCoord(y_points, standard_name="projection_y_coordinate", units="m")
    scalar_t_coord = DimCoord(t_point, standard_name="time", units="hours since epoch")
    dcad = [(y_coord, 0), (x_coord, 1)]
        
    # Construct cube.
    cube = Cube(data, long_name=name, units="K",
                dim_coords_and_dims=dcad)
    cube.add_aux_coord(scalar_t_coord)
    return cube


def make_cubelist(t_points):
    x_points = [0, 1, 2]
    y_points = [0, 1, 2, 3]
    xyt_cube = make_cube(x_points, y_points, t_points)
    
    z_points = [0, 1]
    xyzt_cube = make_cube(x_points, y_points, t_points, z_points)
    
    return CubeList([xyt_cube, xyzt_cube])

## Test Datasets

### Single-dim append

Create two cubes to test single-dim, non-scalar append.

In [ ]:
xpts = [0, 1, 2]
ypts = [0, 1, 2, 3]
tpts0 = [0, 1]
tpts1 = [2, 3]

sdac0 = make_cube(xpts, ypts, tpts0)
sdac1 = make_cube(xpts, ypts, tpts1)

iris.save(sdac0, "xy_t0.nc")
iris.save(sdac1, "xy_t1.nc")

In [ ]:
sdac0

### Multi-dim append

Create four cubes (two for each of two append dims `z` and `t`) to tests multi-dim, all non-scalar append.

In [ ]:
xpts = [0, 1, 2]
ypts = [0, 1, 2, 3]
tpts0 = [0, 1]
tpts1 = [2, 3]
zpts0 = [0, 1, 2]
zpts1 = [3, 4, 5]

mdac0 = make_cube(xpts, ypts, tpts0, z_points=zpts0)
mdac1 = make_cube(xpts, ypts, tpts1, z_points=zpts0)
mdac2 = make_cube(xpts, ypts, tpts0, z_points=zpts1)
mdac3 = make_cube(xpts, ypts, tpts1, z_points=zpts1)

iris.save(mdac0, "xy_t0_z0.nc")
iris.save(mdac1, "xy_t1_z0.nc")
iris.save(mdac2, "xy_t0_z1.nc")
iris.save(mdac3, "xy_t1_z1.nc")

In [ ]:
mdac0

### Scalar append

Create two cubes to test single dim scalar append.

In [ ]:
xpts = [0, 1, 2]
ypts = [0, 1, 2, 3]
tpts0 = 0
tpts1 = 1

sac0 = make_scalar_t_cube(xpts, ypts, tpts0)
sac1 = make_scalar_t_cube(xpts, ypts, tpts1)

iris.save(sac0, "xy_ts0.nc")
iris.save(sac1, "xy_ts1.nc")

In [ ]:
sac0

#### Multiple single-phenomenon datasets

Create two further identical cubes with a different phenomenon name.

In [ ]:
sacw0 = make_scalar_t_cube(xpts, ypts, tpts0, name="wibble")
sacw1 = make_scalar_t_cube(xpts, ypts, tpts1, name="wibble")

iris.save(sacw0, "xy_tws0.nc")
iris.save(sacw1, "xy_tws1.nc")

In [ ]:
sacw0

### Multi-domain append

Create two cubelists of two cubes each to test appends where each TileDB array has multiple domains.

In [ ]:
tpts0 = [0, 1]
tpts1 = [2, 3]

domcl0 = make_cubelist(tpts0)
domcl1 = make_cubelist(tpts1)

iris.save(domcl0, "xy_z01_t0.nc")
iris.save(domcl1, "xy_z01_t1.nc")

In [ ]:
domcl0

## Post-process NetCDFs

`tiledb-netcdf` uses a `coordinates` attribute on `Variable` objects to locate data variables, which Iris does not write, so we recreate the attribute here.

**Note:** need to restart kernel to make this change as Iris will open handles to each written NetCDF file, which prevents them from being re-opened here.

In [ ]:
files = ["xy_t0.nc", "xy_t1.nc",
         "xy_t0_z0.nc", "xy_t1_z0.nc", "xy_t0_z1.nc", "xy_t1_z1.nc",
         "xy_ts0.nc", "xy_ts1.nc",
         "xy_z01_t0.nc", "xy_z01_t1.nc"]
files2 = ["xy_tws0.nc", "xy_tws1.nc"]

In [ ]:
for nc_file in files:
    ncds = netCDF4.Dataset(nc_file, "r+")
    ncds.variables["thingness"].coordinates = " ".join(ncds.variables["thingness"].dimensions)

for nc_file in files2:
    ncds = netCDF4.Dataset(nc_file, "r+")
    ncds.variables["wibble"].coordinates = " ".join(ncds.variables["wibble"].dimensions)